 **TEXT-TO-SQL GENERATOR**

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
# Fix CUDA fragmentation early!
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

!pip install -q bitsandbytes accelerate transformers peft datasets


In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from datasets import Dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model


In [ ]:
model_name = "defog/sqlcoder-7b-2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_enable_fp32_cpu_offload=True,  # this will offload some parts automatically
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Fix missing pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Prepare for k-bit training and add LoRA adapters
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 8,388,608 || all params: 11,603,939,328 || trainable%: 0.0723


In [ ]:
data = {
    "QUESTION": [
        "List all customers who ordered more than 3 times in the last month.",
        "What is the total revenue generated by each merchant?"
    ],
    "QUERY": [
        "SELECT customer_id FROM orders WHERE order_date >= DATE_SUB(CURDATE(), INTERVAL 1 MONTH) GROUP BY customer_id HAVING COUNT(*) > 3;",
        "SELECT merchant_id, SUM(order_total) FROM orders GROUP BY merchant_id;"
    ]
}

dataset = Dataset.from_dict(data)


In [ ]:
def preprocess(example):
    prompt = f"-- Question: {example['QUESTION']}\n-- SQL:\n{example['QUERY']}"
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(preprocess)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
train_dataset = tokenized_dataset.select([0])
eval_dataset = tokenized_dataset.select([1])


In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)


In [ ]:
training_args = TrainingArguments(
    output_dir="./sqlcoder-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    logging_dir="./logs",
    logging_steps=10,
    max_steps=100,
    save_steps=50,
    save_total_limit=2,
    report_to="none",
    fp16=False,
    do_eval=True,
    optim="adamw_torch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

import torch
torch.cuda.empty_cache()

trainer.train()


<ipython-input-53-b2e6532f2be1>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args,

Step,Training Loss
10,1.907200
20,1.771000
30,1.612600
40,1.449100
50,1.299400
60,1.180300
70,1.073900
80,0.997400
90,0.942800
100,0.911300


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=100, training_loss=1.3145036029815673, metrics={'train_runtime': 238.3252, 'train_samples_per_second': 3.357, 'train_steps_per_second': 0.42, 'total_flos': 2032373071872000.0, 'train_loss': 1.3145036029815673, 'epoch': 100.0})

In [ ]:
eval_results = trainer.evaluate()
print("Eval Results:", eval_results)


Eval Results: {'eval_loss': 2.137584686279297, 'eval_runtime': 0.9082, 'eval_samples_per_second': 1.101, 'eval_steps_per_second': 1.101, 'epoch': 100.0}


In [ ]:
import gradio as gr
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model.eval()
model.to(device)

def generate_sql(question, max_length=256):
    prompt = f"-- Question: {question}\n-- SQL:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            do_sample=False,
            num_beams=4,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id
        )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the SQL part from the generated text:
    sql_start = decoded.find("-- SQL:")
    if sql_start != -1:
        return decoded[sql_start + len("-- SQL:"):].strip()
    return decoded.strip()

iface = gr.Interface(
    fn=generate_sql,
    inputs=gr.Textbox(lines=2, placeholder="Enter your natural language question here..."),
    outputs=gr.Textbox(label="Generated SQL Query"),
    title="SQLCoder Fine-tuned - NL to SQL",
    description="Enter a natural language question, and get a generated SQL query."
)

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://51d66a978c5594abdf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
